Before running inference:
1. Run [3.0_Model_conversion_onnx.ipynb](3.0_Model_conversion_onnx.ipynb) to convert .pth model files into .onnx.

In [1]:
import numpy as np
import onnxruntime as ort
import cv2
from PIL import Image
from IPython.display import display
import torchvision.transforms as transforms

In [2]:
def apply_cm(
    image: np.array, map: np.array, threshold: float = 0.5, alpha: float = 0.5
) -> np.array:
    """Apply color map on image

    Args:
        image (np.array): Image
        map (np.array): Grayscale, unnormalized cm.
        threshold (float, optional): Threshold for map. Defaults to 0.5.
        alpha (float, optional): Transparency. Defaults to 0.5.

    Returns:
        np.array: Image with applied cm.
    """
    map_thresholded = np.where(map > threshold, 1, 0)
    map_thresholded = (map_thresholded * 255).astype(np.uint8)
    map_thresholded = cv2.cvtColor(map_thresholded, cv2.COLOR_GRAY2RGB)

    map_normalized = cv2.normalize(
        map, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U
    )
    map_colored = cv2.applyColorMap(map_normalized, cv2.COLORMAP_JET)
    blended_image = cv2.addWeighted(image, 1 - alpha, map_colored, alpha, 0)

    return blended_image

In [3]:
def display_img_with_map(out: np.array, map: np.array, image: np.array) -> None:
    """Displays image with explainability map.

    Args:
        out (np.array): Neural network prediction.
        map (np.array): Neural network explainability output.
        image (np.array): Original inference image.
    """
    preds = out > 0.5
    for i, pred in enumerate(preds):
        map_segmentation = map[i] if pred == 1 else np.zeros_like(map[i])
        if pred == 1:
            map_segmentation = (map_segmentation - map_segmentation.min()) / (
                map_segmentation.max() - map_segmentation.min()
            )
        blended_img = apply_cm(image, map_segmentation)

        # Only for displaying
        blended_img = cv2.cvtColor(blended_img, cv2.COLOR_BGR2RGB)
        display(Image.fromarray(blended_img))

In [ ]:
image = cv2.imread('../docs/sample_data/01_short_04_1926_1070.png', cv2.IMREAD_COLOR)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
display(Image.fromarray(image))
label = cv2.imread(
    '../docs/sample_data/01_short_04_1926_1070_label.png', cv2.IMREAD_GRAYSCALE
)
display(Image.fromarray(label))

In [5]:
transform = transforms.Compose([transforms.ToTensor()])
image_tensor = transform(image).unsqueeze(0)

In [ ]:
# List available execution providers
print('Available execution providers:', ort.get_available_providers())

In [7]:
onnx_file_path = 'efficientnet_v2_s_downscaled_pcb.onnx'
session = ort.InferenceSession(onnx_file_path, providers=['CUDAExecutionProvider'])
input = image_tensor.numpy()
input_name = session.get_inputs()[0].name
output_names = [output.name for output in session.get_outputs()]

In [ ]:
%%timeit
session.run(output_names, {input_name: input})

In [ ]:
outputs = session.run(output_names, {input_name: input})
display_img_with_map(outputs[0], outputs[1], image)

In [14]:
onnx_file_path = 'vit_tiny_patch16_224.augreg_in21k_ft_in1k_pcb.onnx'
session = ort.InferenceSession(onnx_file_path, providers=['CUDAExecutionProvider'])
input = image_tensor.numpy()
input_name = session.get_inputs()[0].name
output_names = [output.name for output in session.get_outputs()]


In [ ]:
%%timeit
session.run(output_names, {input_name: input})

In [ ]:
outputs = session.run(output_names, {input_name: input})
display_img_with_map(outputs[0], outputs[1], image)